<a href="https://colab.research.google.com/github/rohit-mhatre/CS-535-DM/blob/main/dataMining_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

problem 1

In [1]:
import pandas as pd


In [2]:
# Load data from adult.data and adult.test files
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
    "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
    "hours-per-week", "native-country", "income"
]

In [3]:
train_data = pd.read_csv("adult.data", names=columns, na_values=[" ?", "?"])
test_data = pd.read_csv("adult.test", names=columns, na_values=[" ?", "?"])

In [4]:
# Merge the datasets for cleaning
combined_data = pd.concat([train_data, test_data], ignore_index=True)

In [5]:
# Handle missing values
# You can handle missing values using techniques like dropping rows, imputation, etc.
# For simplicity, let's drop rows with missing values in this example
combined_data.dropna(inplace=True)

In [6]:
# Preprocess the data (e.g., convert categorical variables to numerical)
# One-hot encoding for categorical variables
combined_data = pd.get_dummies(combined_data, columns=[
    "workclass", "education", "marital-status", "occupation",
    "relationship", "race", "sex", "native-country"
])

In [7]:
# Convert income column to binary labels (0 for <=50K, 1 for >50K)
combined_data["income"] = combined_data["income"].apply(lambda x: 0 if x == " <=50K" else 1)

In [8]:
# Split data back into train and test sets
train_cleaned = combined_data.iloc[:train_data.shape[0]]
test_cleaned = combined_data.iloc[train_data.shape[0]:]

In [9]:
# Save cleaned data to new files if needed
train_cleaned.to_csv("cleaned_adult_train.csv", index=False)
test_cleaned.to_csv("cleaned_adult_test.csv", index=False)

In [ ]:
print("Data cleaning and preprocessing completed.")


Data cleaning and preprocessing completed.


problem 2:

In [10]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [11]:
# Load cleaned training and evaluation datasets
train_data = pd.read_csv("cleaned_adult_train.csv")
eval_data = pd.read_csv("cleaned_adult_test.csv")

In [13]:
# Separate features and labels
X_train = train_data.drop(columns=["income"])
y_train = train_data["income"]
X_eval = eval_data.drop(columns=["income"])
y_eval = eval_data["income"]

In [14]:
# Initialize and train Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [15]:
# Predict on evaluation dataset
y_pred = rf_classifier.predict(X_eval)

In [16]:
# Calculate classification error rate
classification_error_rate = 1 - accuracy_score(y_eval, y_pred)


In [17]:
# Print classification error rate
print("Classification Error Rate:", classification_error_rate)

Classification Error Rate: 0.7628939262301556


problem 4:

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
# Load cleaned evaluation dataset
eval_data = pd.read_csv("cleaned_adult_test.csv")
X_eval = eval_data.drop(columns=["income"])
y_eval = eval_data["income"]

In [ ]:
# Define sampling rates
sampling_rates = [50, 60, 70, 80, 90]

In [ ]:
# Initialize lists to store error rates
error_rates = []

In [ ]:
# Repeat the process 5 times for each sampling rate
for rate in sampling_rates:
    error_rates_for_rate = []
    for _ in range(5):  # Repeat 5 times
        # Load cleaned training dataset
        train_data = pd.read_csv("cleaned_adult_train.csv")

        # Separate samples by income class
        low_income_samples = train_data[train_data['income'] == 0].sample(frac=(rate/100))
        high_income_samples = train_data[train_data['income'] == 1].sample(frac=(rate/100))

        # Concatenate samples and shuffle
        downsampled_train_data = pd.concat([low_income_samples, high_income_samples]).sample(frac=1)

        # Separate features and labels
        X_train = downsampled_train_data.drop(columns=["income"])
        y_train = downsampled_train_data["income"]

        # Initialize and train Random Forest classifier
        rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
        rf_classifier.fit(X_train, y_train)

        # Predict on evaluation dataset
        y_pred = rf_classifier.predict(X_eval)

        # Calculate error rate
        error_rate = 1 - accuracy_score(y_eval, y_pred)
        error_rates_for_rate.append(error_rate)

    # Store mean and standard deviation for each rate
    error_rates.append((rate, np.mean(error_rates_for_rate), np.std(error_rates_for_rate)))

In [ ]:
# Print results
print("Sampling Rate\tMean Error Rate\tStandard Deviation")
for rate, mean, std_dev in error_rates:
    print(f"{rate}%\t\t{mean:.6f}\t\t{std_dev:.6f}")

Sampling Rate	Mean Error Rate	Standard Deviation
50%		0.766796		0.003699
60%		0.765342		0.004424
70%		0.765358		0.001040
80%		0.765311		0.001841
90%		0.766101		0.000775


problem 5:

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import accuracy_score

In [ ]:

# Load cleaned training and evaluation datasets
train_data = pd.read_csv("cleaned_adult_train.csv")
eval_data = pd.read_csv("cleaned_adult_test.csv")

In [ ]:
# Separate features and labels
X_train = train_data.drop(columns=["income"])
y_train = train_data["income"]
X_eval = eval_data.drop(columns=["income"])
y_eval = eval_data["income"]

In [ ]:
# Initialize and train LightGBM classifier
gbm_classifier = lgb.LGBMClassifier(num_leaves=31, learning_rate=0.05, n_estimators=100)
gbm_classifier.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 9907, number of negative: 22654
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016806 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 769
[LightGBM] [Info] Number of data points in the train set: 32561, number of used features: 93
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.304260 -> initscore=-0.827095
[LightGBM] [Info] Start training from score -0.827095


LGBMClassifier(learning_rate=0.05)

In [ ]:
# Predict on evaluation dataset
y_pred = gbm_classifier.predict(X_eval)

In [ ]:
# Calculate classification error rate
classification_error_rate = 1 - accuracy_score(y_eval, y_pred)


In [ ]:
# Print classification error rate
print("Classification Error Rate:", classification_error_rate)

Classification Error Rate: 0.779164363004502
